In [1]:
import json
import pandas as pd

# Salto

In [34]:
with open('../data/data_salto/concursos/concursos_salto_internacional_2020.json', 'r', encoding='utf-8') as file:
    datos_pruebas = json.load(file)

# Ver la longitud de cada lista dentro del JSON
for key, value in datos_pruebas.items():
    if isinstance(value, list):  # Verificamos si el valor es una lista
        print(f"La longitud de {key} es: {len(value)}")

La longitud de Nombre es: 1936
La longitud de Categoría es: 1936
La longitud de Provincia es: 1936
La longitud de Localidad es: 1936
La longitud de Disciplina es: 1936
La longitud de Federación es: 1936
La longitud de Resultados es: 1936
La longitud de País es: 1936
La longitud de Inicio es: 1935
La longitud de Final es: 1935
La longitud de Ambito es: 1935


In [35]:
with open('../data/data_salto/pruebas/pruebas_salto_internacional_2020.json', 'r', encoding='utf-8') as file:
    datos_pruebas = json.load(file)

# Ver la longitud de cada lista dentro del JSON
for key, value in datos_pruebas.items():
    if isinstance(value, list):  # Verificamos si el valor es una lista
        print(f"La longitud de {key} es: {len(value)}")

La longitud de Disciplina es: 14979
La longitud de Fecha es: 14970
La longitud de Prueba es: 14957
La longitud de Categoría es: 14808
La longitud de Número es: 14797
La longitud de Concurso es: 14979


In [36]:
with open('../data/data_salto/concursos/concursos_salto_nacional_2020.json', 'r', encoding='utf-8') as file:
    datos_pruebas = json.load(file)

# Ver la longitud de cada lista dentro del JSON
for key, value in datos_pruebas.items():
    if isinstance(value, list):  # Verificamos si el valor es una lista
        print(f"La longitud de {key} es: {len(value)}")

La longitud de Nombre es: 1300
La longitud de Categoría es: 1300
La longitud de Provincia es: 1300
La longitud de Localidad es: 1300
La longitud de Disciplina es: 1300
La longitud de Federación es: 1300
La longitud de Resultados es: 1300
La longitud de País es: 1300
La longitud de Inicio es: 1296
La longitud de Final es: 1296
La longitud de Ambito es: 1296


In [37]:
with open('../data/data_salto/pruebas/pruebas_salto_nacional_2020.json', 'r', encoding='utf-8') as file:
    datos_pruebas = json.load(file)

# Ver la longitud de cada lista dentro del JSON
for key, value in datos_pruebas.items():
    if isinstance(value, list):  # Verificamos si el valor es una lista
        print(f"La longitud de {key} es: {len(value)}")

La longitud de Disciplina es: 13393
La longitud de Fecha es: 13393
La longitud de Prueba es: 13393
La longitud de Categoría es: 13393
La longitud de Número es: 13393
La longitud de Concurso es: 13393


# Completo

In [38]:
with open('../data/data_completo/pruebas/pruebas_completo_nacional_2020.json', 'r', encoding='utf-8') as file:
    datos_pruebas = json.load(file)

# Ver la longitud de cada lista dentro del JSON
for key, value in datos_pruebas.items():
    if isinstance(value, list):  # Verificamos si el valor es una lista
        print(f"La longitud de {key} es: {len(value)}")

La longitud de Disciplina es: 561
La longitud de Fecha es: 561
La longitud de Prueba es: 561
La longitud de Categoría es: 561
La longitud de Número es: 561
La longitud de Concurso es: 561


In [39]:
with open('../data/data_completo/pruebas/pruebas_completo_internacional_2020.json', 'r', encoding='utf-8') as file:
    datos_pruebas = json.load(file)

# Ver la longitud de cada lista dentro del JSON
for key, value in datos_pruebas.items():
    if isinstance(value, list):  # Verificamos si el valor es una lista
        print(f"La longitud de {key} es: {len(value)}")

La longitud de Disciplina es: 496
La longitud de Fecha es: 496
La longitud de Prueba es: 496
La longitud de Categoría es: 496
La longitud de Número es: 496
La longitud de Concurso es: 496


In [40]:
with open('../data/data_completo/concursos/concursos_completo_internacional_2020.json', 'r', encoding='utf-8') as file:
    datos_pruebas = json.load(file)

# Ver la longitud de cada lista dentro del JSON
for key, value in datos_pruebas.items():
    if isinstance(value, list):  # Verificamos si el valor es una lista
        print(f"La longitud de {key} es: {len(value)}")

La longitud de Nombre es: 480
La longitud de Categoría es: 480
La longitud de Provincia es: 480
La longitud de Localidad es: 480
La longitud de Disciplina es: 480
La longitud de Federación es: 480
La longitud de Resultados es: 480
La longitud de País es: 480
La longitud de Inicio es: 477
La longitud de Final es: 477
La longitud de Ambito es: 477


In [41]:
with open('../data/data_completo/concursos/concursos_completo_nacional_2020.json', 'r', encoding='utf-8') as file:
    datos_pruebas = json.load(file)

# Ver la longitud de cada lista dentro del JSON
for key, value in datos_pruebas.items():
    if isinstance(value, list):  # Verificamos si el valor es una lista
        print(f"La longitud de {key} es: {len(value)}")

La longitud de Nombre es: 223
La longitud de Categoría es: 223
La longitud de Provincia es: 223
La longitud de Localidad es: 223
La longitud de Disciplina es: 223
La longitud de Federación es: 223
La longitud de Resultados es: 223
La longitud de País es: 223
La longitud de Inicio es: 223
La longitud de Final es: 223
La longitud de Ambito es: 223
